In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [2]:
# Load and preprocess CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
cnn_model = models.Sequential([
    layers.Input(shape=(32, 32, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [6]:
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.6504 - loss: 0.9910 - val_accuracy: 0.6576 - val_loss: 0.9929
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.6823 - loss: 0.8997 - val_accuracy: 0.6776 - val_loss: 0.9408
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.7131 - loss: 0.8182 - val_accuracy: 0.6885 - val_loss: 0.8995
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.7368 - loss: 0.7557 - val_accuracy: 0.6997 - val_loss: 0.8819
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.7501 - loss: 0.7042 - val_accuracy: 0.6939 - val_loss: 0.9089
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.7677 - loss: 0.6567 - val_accuracy: 0.7089 - val_loss: 0.8704
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.7817 - loss: 0.6216 - val_accuracy: 0.7113 - val_loss: 0.8727
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.7988 - loss: 0

In [9]:
# extract features except the last output layer
fe = models.Model(inputs=cnn_model.inputs, outputs=cnn_model.layers[-2].output)
x_train_features = fe.predict(x_train)
x_test_features = fe.predict(x_test)


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [12]:
# Random Forest on the extracted features
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train_features, y_train.ravel())
rf_accuracy = accuracy_score(y_test.ravel(), rf.predict(x_test_features))
print(f'Random Forest accuracy: {rf_accuracy}')
print(f'CNN accuracy: {cnn_model.evaluate(x_test, y_test)[1]}')

Random Forest accuracy: 0.7148
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7209 - loss: 0.8770
CNN accuracy: 0.7218000292778015
